In [135]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

In [136]:
pd.set_option('display.max_colwidth', 250)

In [137]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%B8%D6%82%D5%B6%D5%AF,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1789,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1790,/wiki/%D5%9B,՛
1791,/wiki/%D5%9E,՞
1792,/wiki/%D6%89,։


In [138]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

In [139]:
def getHeadingLevel(cs):
    levels = [c for c in cs if c != 'mw-heading']
    if len(levels) > 0:
        return levels[0]

In [140]:
def getSubSections(section):
    results = []
    result = {}
    current_header = None
    current_subsection = bs4.Tag(name='div')
    top_heading_level = None
    for tag in section:
        classes = tag.attrs.get('class', [])
        if tag.name == 'div' and 'mw-heading' in classes:
            heading_level = getHeadingLevel(classes)
            if not top_heading_level:
                top_heading_level = heading_level
            elif heading_level == top_heading_level and result:
                results.append(result)
                result = {}
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = bs4.Tag(name='div')
        else:
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    results.append(result)
    return results

In [172]:
entries = {}

for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.sample(1).iterrows()]):
#for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas[lemmas.title.isin(['աման', 'փիս', '-անք', 'համ'])].iterrows()]):
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


In [173]:
pd.DataFrame([dict(title=k, idx=i+1, **r) for k,v in entries.items() for i,r in enumerate(v)]).set_index(['title', 'idx'])

Alternative forms  \
title idx                                                                                                                                                                                                                                                              
ծանր  1    [[[<span class="Armn" lang="hy"><a class="new" href="/w/index.php?title=%D5%AE%D5%A1%D5%B6%D5%A4%D6%80&amp;action=edit&amp;redlink=1" title="ծանդր (page does not exist)">ծանդր</a></span>,  , <span class="mention-gloss-paren annotation-paren">(</s...   
      2                                                                                                                                                                                                                                                          NaN   
      3                                                                                                                                                                                                                                                          NaN   

                                                                                                                                                                                                                                                 Etymology  \
title idx                                                                                                                                                                                                                                                    
ծանր  1                                                                                                                                                                                                                                                NaN   
      2    [[[Inherited],  from , [<a class="extiw" href="https://en.wikipedia.org/wiki/Classical_Armenian" title="w:Classical Armenian">Old Armenian</a>],  , [<a class="mw-selflink-fragment" href="#Old_Armenian">ծանր</a>],  , [(], [canr], [)], .\n]]   
      3                                                                                                                                                                                                                                                NaN   

                                                                                                                                                                                                                                                       Pronunciation  \
title idx                                                                                                                                                                                                                                                              
ծանր  1                                                                                                                                                                                                                                                          NaN   
      2                                                                                                                                                                                                                                                          NaN   
      3    [[[], \n, [<span class="usage-label-accent"><span class="ib-brac">(</span><span class="ib-content"><a class="extiw" href="https://en.wikipedia.org/wiki/Eastern_Armenian" title="w:Eastern Armenian">Eastern Armenian</a></span><span class="ib-brac">...   

                                                                                                                                                                                                                                                           Adjective  \
title idx                                                         

In [142]:
pd.DataFrame(entries)#]

,աման,-անք,համ,փիս
0,"{'Pronunciation': [[<li class=""mw-empty-elt""></li>, ' ', <li><span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian""...","{'Pronunciation': [[<li class=""mw-empty-elt""></li>, ' ', <li><span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian""...","{'Pronunciation': [[<li class=""mw-empty-elt""></li>, ' ', <li><span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian""...","{'Pronunciation': [[<li class=""mw-empty-elt""></li>, ' ', <li><span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian""..."
1,"{'Descendants': [[<li><span class=""desc-arr"" title=""borrowed"">→</span> Northern Kurdish: <span class=""Latn"" lang=""kmr""><a href=""/wiki/aman#Northern_Kurdish"" title=""aman"">aman</a></span><style data-mw-deduplicate=""TemplateStyles:r68481116"">.mw-par...","{'Derived terms': [[<div class=""CategoryTreeSection""><div class=""CategoryTreeItem""><span class=""CategoryTreeBullet""><a aria-expanded=""false"" class=""CategoryTreeToggle"" data-ct-title=""Armenian_terms_suffixed_with_-անք_(collective)""></a> </span> <a...","{'Derived terms': [[<li><span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%B0%D5%A1%D5%B4%D5%B6_%D5%A1%D5%BC%D5%B6%D5%A5%D5%AC&amp;action=edit&amp;redlink=1"" title=""համն առնել (page does not exist)"">համն առնել</a></span> <sp...","{'Further reading': [[<li><span class=""citation-whole""><span class=""cited-source""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Stepan_Malkhasyants"" title=""w:Stepan Malkhasyants"">Malxaseancʻ, Stepʻan</a> (<span class=""None"" lang=""und"">1945..."


In [143]:
results = pd.DataFrame(entries).T

In [144]:
columns_by_freq = results.count().sort_values(ascending=False)
results = results[columns_by_freq.index] #[lambda x:x['Verb'].notna()]
columns_by_freq.to_frame().style.bar()

,0
0,4
1,4


In [145]:
def process_pronunciation(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return '\n'.join([li.text for li in tag.select('li') if not 'Audio' in li.text and not li.text == ''])

results['Pronunciation'] = results['Pronunciation'].apply(process_pronunciation)

KeyError: 'Pronunciation'

In [ ]:
def process_basic_text(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return tag.text.strip()


results['Etymology'] = results['Etymology'].apply(process_basic_text)
results['Etymology 1'] = results['Etymology 1'].apply(process_basic_text)
results['Etymology 2'] = results['Etymology 2'].apply(process_basic_text)

KeyError: 'Etymology'

In [ ]:
results

,Pronunciation,Etymology 1,Declension,Derived terms,Etymology 2,Noun,Alternative forms,Further reading,Descendants,Interjection,Suffix,References,Conjunction,Usage notes,Adjective,Adverb
աման,,,[],[],,[],NaN,NaN,[],[],NaN,NaN,NaN,NaN,NaN,NaN
-անք,,,[],[],,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
համ,,,[],[],,[],[],[],NaN,NaN,NaN,NaN,[],[],NaN,NaN
փիս,,,[],[],,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [ ]:
results[results['Etymology 2'].notna()].T.columns

Index(['աման', '-անք', 'համ', 'փիս'], dtype='object')